Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-horovod/distributed-pytorch-with-horovod.png)

# Distributed PyTorch with DistributedDataParallel

In this tutorial, you will train a PyTorch model on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset using distributed training with PyTorch's `DistributedDataParallel` module across a GPU cluster.

## 前提条件確認 Prerequisites
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [Configuration](../../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## ワークスペースの設定 Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.  
Azure ML Studioから構成ファイル (config.json)をダウンロードし、本スクリプトと同一階層に置きます。

In [4]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: ml-lab
Azure region: westus2
Subscription id: f57ce3c6-5c6f-4f1e-8cba-b782d8974590
Resource group: rg-aml


## 計算環境の準備 Create or attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Specifically, the below code creates an `STANDARD_NC6` GPU cluster that autoscales from `0` to `4` nodes.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace, this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = 'gpu-cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-22T09:03:03.412000+00:00', 'errors': None, 'creationTime': '2021-03-22T09:03:01.130767+00:00', 'modifiedTime': '2021-03-22T09:03:16.578884+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


The above code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## データセットの準備 Prepare dataset

Prepare the dataset used for training. We will first download and extract the publicly available CIFAR-10 dataset from the cs.toronto.edu website and then create an Azure ML FileDataset to use the data for training.

### Download and extract CIFAR-10 data

In [6]:
import urllib
import tarfile
import os

url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
filename = 'cifar-10-python.tar.gz'
data_root = 'cifar-10'
filepath = os.path.join(data_root, filename)

if not os.path.isdir(data_root):
    os.makedirs(data_root, exist_ok=True)
    urllib.request.urlretrieve(url, filepath)
    with tarfile.open(filepath, "r:gz") as tar:
        tar.extractall(path=data_root)
    os.remove(filepath)  # delete tar.gz file after extraction

### Create Azure ML dataset

The `upload_directory` method will upload the data to a datastore and create a FileDataset from it. In this tutorial we will use the workspace's default datastore.

In [9]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
dataset = Dataset.File.upload_directory(
    src_dir=data_root, target=(datastore, data_root)
)

Method upload_directory: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Validating arguments.
Arguments validated.
Uploading file to cifar-10
Uploading an estimated of 8 files
Uploading cifar-10/cifar-10-batches-py/batches.meta
Uploaded cifar-10/cifar-10-batches-py/batches.meta, 1 files out of an estimated total of 8
Uploading cifar-10/cifar-10-batches-py/readme.html
Uploaded cifar-10/cifar-10-batches-py/readme.html, 2 files out of an estimated total of 8
Uploading cifar-10/cifar-10-batches-py/data_batch_4
Uploaded cifar-10/cifar-10-batches-py/data_batch_4, 3 files out of an estimated total of 8
Uploading cifar-10/cifar-10-batches-py/data_batch_5
Uploaded cifar-10/cifar-10-batches-py/data_batch_5, 4 files out of an estimated total of 8
Uploading cifar-10/cifar-10-batches-py/data_batch_3
Uploaded cifar-10/cifar-10-batches-py/data_batch_3, 5 files out of an estimated total of 8
Uploading cifar-10/cifar-10-batc

## モデル学習 Train model on the remote compute
Now that we have the AmlCompute ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [10]:
project_folder = './pytorch-distr'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training on CIFAR-10 is already provided for you at `train.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

Once your script is ready, copy the training script `train.py` into the project directory.
トレーニングスクリプトをプロジェクトディレクトリ内へコピーします。

In [11]:
import shutil

shutil.copy('train.py', project_folder)

'./pytorch-distr/train.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 
実験を設定します。

In [12]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

In this tutorial, we will use one of Azure ML's curated PyTorch environments for training. [Curated environments](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments#use-a-curated-environment) are available in your workspace by default. Specifically, we will use the PyTorch 1.6 GPU curated environment.  

Azure MLではいくつかの[キュレートされた実行環境](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-environments#use-a-curated-environment)が用意されています。
今回はPyTorch 1.6 GPU環境を使用します。こちらのキュレートされた環境には今回のトレーニングスクリプトで必要なtorch, torchvisionも含まれています。

参考：[キュレーションされた環境一覧](https://docs.microsoft.com/ja-jp/azure/machine-learning/resource-curated-environments)

In [13]:
from azureml.core import Environment

pytorch_env = Environment.get(ws, name='AzureML-PyTorch-1.6-GPU')

### Configure the training job

To launch a distributed PyTorch job on Azure ML, you have two options:

1. Per-process launch - specify the total # of worker processes (typically one per GPU) you want to run, and
Azure ML will handle launching each process.
2. Per-node launch with [torch.distributed.launch](https://pytorch.org/docs/stable/distributed.html#launch-utility) - provide the `torch.distributed.launch` command you want to
run on each node.

For more information, see the [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-pytorch#distributeddataparallel).

Both options are shown below.

#### Per-process launch

To use the per-process launch option in which Azure ML will handle launching each of the processes to run your training script,

1. Specify the training script and arguments
2. Create a `PyTorchConfiguration` and specify `node_count` and `process_count`. The `process_count` is the total number of processes you want to run for the job; this should typically equal the # of GPUs available on each node multiplied by the # of nodes. Since this tutorial uses the `STANDARD_NC6` SKU, which has one GPU, the total process count for a 2-node job is `2`. If you are using a SKU with >1 GPUs, adjust the `process_count` accordingly.

Azure ML will set the `MASTER_ADDR`, `MASTER_PORT`, `NODE_RANK`, `WORLD_SIZE` environment variables on each node, in addition to the process-level `RANK` and `LOCAL_RANK` environment variables, that are needed for distributed PyTorch training.

In [14]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

# create distributed config
distr_config = PyTorchConfiguration(process_count=2, node_count=2)

# create args
args = ["--data-dir", dataset.as_download(), "--epochs", 25]

# create job config
src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=pytorch_env,
                      distributed_job_config=distr_config)

#### Per-node launch with `torch.distributed.launch`

If you would instead like to use the PyTorch-provided launch utility `torch.distributed.launch` to handle launching the worker processes on each node, you can do so as well. 

1. Provide the launch command to the `command` parameter of ScriptRunConfig. For PyTorch jobs Azure ML will set the `MASTER_ADDR`, `MASTER_PORT`, and `NODE_RANK` environment variables on each node, so you can simply just reference those environment variables in your command. If you are using a SKU with >1 GPUs, adjust the `--nproc_per_node` argument accordingly.

2. Create a `PyTorchConfiguration` and specify the `node_count`. You do not need to specify the `process_count`; by default Azure ML will launch one process per node to run the `command` you provided.

Uncomment the code below to configure a job with this method.

In [ ]:
'''
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

# create distributed config
distr_config = PyTorchConfiguration(node_count=2)

# define command
launch_cmd = ["python -m torch.distributed.launch --nproc_per_node 1 --nnodes 2 " \
    "--node_rank $NODE_RANK --master_addr $MASTER_ADDR --master_port $MASTER_PORT --use_env " \
    "train.py --data-dir", dataset.as_download(), "--epochs 25"]

# create job config
src = ScriptRunConfig(source_directory=project_folder,
                      command=launch_cmd,
                      compute_target=compute_target,
                      environment=pytorch_env,
                      distributed_job_config=distr_config)
'''

### Submit job
Run your experiment by submitting your `ScriptRunConfig` object. Note that this call is asynchronous.

In [15]:
run = experiment.submit(src)
print(run)

Run(Experiment: pytorch-distr,
Id: pytorch-distr_1616469757_10343fff,
Type: azureml.scriptrun,
Status: Preparing)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes. You can see that the widget automatically plots and visualizes the loss metric that we logged to the Azure ML run.

Jupyterウィジェットを使って実行の進捗状況を監視することができます。実行のサブミッションと同様に、ウィジェットは非同期で、ジョブが完了するまで10～15秒ごとに自動で更新されます。ウィジェットでは、Azure MLの実行に記録した損失指標が自動的に表示・可視化されます。

※VSCode上で実行する場合、テーマ設定 (背景色)によってはAzure MLウィジェットが見えにくくなる可能性があります。その場合はLightテーマの使用をお勧めします。

In [16]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Alternatively, you can block until the script has completed training before running more code.

In [17]:
run.wait_for_completion(show_output=True) # this provides a verbose log

RunId: pytorch-distr_1616469757_10343fff
Web View: https://ml.azure.com/experiments/pytorch-distr/runs/pytorch-distr_1616469757_10343fff?wsid=/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-aml/workspaces/ml-lab

Execution Summary
RunId: pytorch-distr_1616469757_10343fff
Web View: https://ml.azure.com/experiments/pytorch-distr/runs/pytorch-distr_1616469757_10343fff?wsid=/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-aml/workspaces/ml-lab



{'runId': 'pytorch-distr_1616469757_10343fff',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-23T03:26:20.712897Z',
 'endTimeUtc': '2021-03-23T03:39:42.480751Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'c0226cb2-4817-4968-8d05-686acc882c88',
  'azureml.git.repository_uri': 'git@github.com:shohei1029/azureml_distributed-pytorch.git',
  'mlflow.source.git.repoURL': 'git@github.com:shohei1029/azureml_distributed-pytorch.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '84044cc0fe460472a82c57dc72329f82d8ab098f',
  'mlflow.source.git.commit': '84044cc0fe460472a82c57dc72329f82d8ab098f',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '5552c1f0-d54d-4c12-9dea-c3c6d08c5c4f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__

## モデルの登録


In [22]:
#実行に関係しているファイル一覧の表示
for i in run.get_file_names():
    print(i)

azureml-logs/55_azureml-execution-tvmps_4f13e3e147a6a49a8fc39a6306f5d937293ceb882e0caaf97da18838e9af81b1_d.txt
azureml-logs/55_azureml-execution-tvmps_b9f912d8d6d3d93bc36ffdeb087aae97569c4c6a2acd31376ef2b5fecf92fc95_d.txt
azureml-logs/65_job_prep-tvmps_4f13e3e147a6a49a8fc39a6306f5d937293ceb882e0caaf97da18838e9af81b1_d.txt
azureml-logs/65_job_prep-tvmps_b9f912d8d6d3d93bc36ffdeb087aae97569c4c6a2acd31376ef2b5fecf92fc95_d.txt
azureml-logs/70_driver_log_0.txt
azureml-logs/70_driver_log_1.txt
azureml-logs/75_job_post-tvmps_4f13e3e147a6a49a8fc39a6306f5d937293ceb882e0caaf97da18838e9af81b1_d.txt
azureml-logs/75_job_post-tvmps_b9f912d8d6d3d93bc36ffdeb087aae97569c4c6a2acd31376ef2b5fecf92fc95_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/0_109_azureml.log
logs/azureml/1_88_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
logs/azure

In [24]:
model = run.register_model(model_name = 'pytorch-distr', model_path = 'outputs/cifar_net.pt')
print(model.name, model.id, model.version, sep = '\t')

pytorch-distr	pytorch-distr:1	1


## モデルデプロイ
Webサービスとしてモデルをデプロイします。

### スコアリングスクリプトの作成
Web サービスの呼び出しに使用される score.py というスコアリング スクリプトを作成してモデルの使用方法を示します。
スコアリング スクリプトには、2 つの必要な関数を含める必要があります。
- `init()` 関数。通常、グローバル オブジェクトにモデルを読み込みます。 この関数は、Docker コンテナーを開始するときに 1 回だけ実行されます。
- `run(input_data)` 関数。モデルを使用して、入力データに基づく値を予測します。 実行に対する入力と出力は、通常、JSON を使用してシリアル化およびシリアル化解除が実行されますが、その他の形式もサポートされています。

In [26]:
%%writefile score.py
import os
import torch
import torch.nn as nn
from torchvision import transforms
import json

from azureml.core.model import Model


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'cifar_net.pt')
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.eval()


def run(input_data):
    input_data = torch.tensor(json.loads(input_data)['data'])

    # get prediction
    with torch.no_grad():
        output = model(input_data)
        classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        softmax = nn.Softmax(dim=1)
        pred_probs = softmax(output).numpy()[0]
        index = torch.argmax(output, 1)

    result = {"label": classes[index], "probability": str(pred_probs[index])}
    return result

Writing score.py


### ACIコンテナへのデプロイ
デプロイの構成ファイルを作成し、ACI コンテナーに必要な CPU 数と RAM ギガバイト数を指定します。 実際のモデルにもよりますが、通常、多くのモデルには既定値の 1 コアと 1 ギガバイトの RAM で十分です。 後でもっと必要になった場合は、イメージを再作成し、サービスをデプロイし直す必要があります。
※今回はデプロイ先の実行環境にはトレーニング時と同一の環境 (pytorch_env)を使用

In [38]:
%%time
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model

inference_config = InferenceConfig(entry_script="score.py", environment=pytorch_env)

#デプロイの構成設定
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'data': 'cifar-10',  'model':'pytorch-distr', 'framework':'pytorch'},
                                               description='Classify daily objects from the cifar-10 dataset using PyTorch')

model = Model(ws, 'pytorch-distr')

service = Model.deploy(workspace=ws, 
                           name='aci-cifar10', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-23 17:15:59+09:00 Creating Container Registry if not exists.
2021-03-23 17:16:00+09:00 Registering the environment.
2021-03-23 17:16:00+09:00 Use the existing image.
2021-03-23 17:16:00+09:00 Generating deployment configuration.
2021-03-23 17:16:01+09:00 Submitting deployment to compute.
2021-03-23 17:16:05+09:00 Checking the status of deployment aci-cifar10..
2021-03-23 17:22:24+09:00 Checking the status of inference endpoint aci-cifar10.
Failed
Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: b134b8d6-a721-4870-8af1-f31c8f0cbc60
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: b134b8d6-a721-4870-8af1-f31c8f0cbc60
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: aci-cifar10. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_31eff05d9938cbbdf21dfb15bccdb84f locally. Please refer to https://aka.ms/

In [39]:
# デプロイ中に問題が発生した場合にログ取得
service.get_logs()

In [ ]:
# 再デプロイ前に既存のACIサービスを削除
service.delete()


## モデルのテスト
